In [ ]:
import os
import ee
import geemap
import pandas as pd
import csv
import re
from pprint import pprint

#laods a blank google maps map
Map = geemap.Map(basemap='SATELLITE')
Map

In [2]:
Map.setCenter(102.6283, 39.404, 19)


In [ ]:
out_dir = os.path.join(r"E:\Grassland Mapping files\Updated Code Outputs")

with open(r"E:\Grassland Mapping Files\coordinateSubmissions.csv", newline='') as file:
    reader = csv.reader(file)
    sites = list(map(tuple, reader))

#create code that will loop once for each of the sites
i = 1
for i in range (500)[1:]:
    #define the name, latitude and longitude from each site
    name = (sites[i][0])
    lat_ = (sites[i][1])
    long_ = (sites[i][2])
    
    #convert the string into a float value
    lat = float(lat_)
    long = float(long_)
    
    #create a region of interest based on the provided lat and long
    lat_1 = (lat - 0.04497)
    lat_2 = (lat + 0.04497)
    long_1 = (long - 0.04497)
    long_2 = (long + 0.04497)

    #alters the coordinates to make a smaller box around the site (these values make a 10x10 box of cells)
    grid_lat_1 = (lat - 0.000405)
    grid_lat_2 = (lat + 0.000405)
    grid_long_1 = (long - 0.000405)
    grid_long_2 = (long + 0.000405)
    
    #creation of a general geometry, just a large area around the site
    roi = ee.Geometry.BBox(long_1, lat_1, long_2, lat_2)
    
    #smaller area, 10x10m grid
    grid_roi = ee.Geometry.BBox(grid_long_1, grid_lat_1, grid_long_2, grid_lat_2)
    
    
    #################################################################
    #create grids around the coordinates of the site
    #################################################################
    # imagery is going to be more tricky - can just use sentinel but makes it even more difficult to differentiate 
    grid_10m = grid_roi.coveringGrid('EPSG:4326', 10)
    
    out_shp = os.path.join(out_dir, name + ' Site.geojson')
    geemap.ee_export_vector(grid_10m, out_shp, verbose=True)
    
    print(i)
    print("...Done")
    i = i + 1

In [ ]:
#################################################################
#generate random coordinates based on the locations provided by experts
#################################################################
#choose out directory
out_dir = os.path.join(r"E:\Grassland Mapping files\Updated Code Outputs")

#load the ESA WC dataset
esa = ee.ImageCollection("ESA/WorldCover/v200").first()
esa_aligned = esa.reproject('EPSG:4326',None, 10)

#mask areas so that only coordinates with grassland cells can be selected. 
grassland = esa_aligned.eq(30)
grasses = esa_aligned.mask(grassland)
Map.addLayer(grasses, {}, 'grassland')

with open(r"E:\Work Files\coordinateSubmissions.csv", newline='') as file:
    reader = csv.reader(file)
    sites = list(map(tuple, reader))

#create code that will loop once for each of the provided sites
i = 1
for i in range (500)[1:]:
    #define the name, latitude and longitude from each site
    name = (sites[i][0])
    lat_ = (sites[i][1])
    long_ = (sites[i][2])
    
    #convert the string into a float value
    lat = float(lat_)
    long = float(long_)
    
    Map.setCenter(long, lat, 10)
    
    #create a region of interest based on the provided lat and long
    #0.04497 = 5km
    lat_roi_1 = (lat - 0.04497)
    lat_roi_2 = (lat + 0.04497)
    long_roi_1 = (long - 0.04497)
    long_roi_2 = (long + 0.04497)
    roi = ee.Geometry.BBox(long_roi_1, lat_roi_1, long_roi_2, lat_roi_2)
    
    #vectorize the roi and add it to the map
    vectors = grasses.addBands(esa_aligned).reduceToVectors(ee.Reducer.mean(), roi, 1, 'polygon', False, 'label', 'EPSG:4326', None, False, maxPixels = 1e10)
    #Map.addLayer(vectors, {min: 0, max: 1, 'pallete': ['0000FF', '00FF00']}, "Vectors")
    
    print('randomizing points')
    points = grasses.sample(vectors, 10, 'EPSG:4326', None, 10, 0, False, None, True)
    #display on map
    Map.addLayer(points, {'color':'red'}, 'randomized points')
    print('points collected')
    
    #convert feature class into a multipoint geometry
    multipoint = points.geometry()
    print('points converted')
    #convert multipoint into a list of lists
    coordinatesList = multipoint.coordinates()
    print(coordinatesList.getInfo())
    
    #export the points for each site
    out_shp = os.path.join(out_dir, name + ' Random Sites.geojson')
    geemap.ee_export_vector(points, out_shp, verbose = True)

    #Loop through the randomly created coordinates, make grids around them and then export them
    j = 0 
    for j in range (10):
        
        #convert list of lists into an individual list of lat/long
        coordinates = coordinatesList.get(j).getInfo()
        
        #get coordinats from  list
        long_new = (coordinates[0])
        lat_new = (coordinates[1])
        
        #convert strings into float values
        latNew = float(lat_new)
        longNew = float(long_new)
        Map.setCenter(longNew, latNew, 17)
        
        var1 = 0.00081/2
        grid_lat_1 = (latNew - var1)
        grid_lat_2 = (latNew + var1)
        grid_long_1 = (longNew - var1)
        grid_long_2 = (longNew + var1)
        
        #create a 10x10 100m grid around the coordinates
        grid_roi = ee.Geometry.BBox(grid_long_1, grid_lat_1, grid_long_2, grid_lat_2)
        grid_10m = grid_roi.coveringGrid('EPSG:4326', 10)
        #Map.addLayer(grid_10m, {}, 'grid'+ str(j))
        print('exporting site grid')
        #export the grids using individual file names
        out_shp = os.path.join(out_dir,name + ' Random Site ' + str(j) + '.geojson')
        geemap.ee_export_vector(grid_10m, out_shp, verbose = True)
        
        j = j + 1
         
    i = i + 1